# day 16

https://adventofcode.com/2019/day/16

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day16.txt')

LOGGER = logging.getLogger('day16')

## part 1

### problem statement:

#### loading data

In [ ]:
test_0 = [int(_) for _ in "80871224585914546619083218645595"]
test_1 = [int(_) for _ in "19617804207202209144916044189917"]
test_2 = [int(_) for _ in "69317163492948606335995924319873"]

a0 = 24176176
a1 = 73745418
a2 = 52432133

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return [int(_) for _ in fp.read().strip()]

In [ ]:
import numpy as np
import tqdm.autonotebook as tqdm

def pattern_array(pattern, L):
    z = np.zeros((L, L + 1), dtype=np.int8)
    
    for i in tqdm.trange(L, leave=False):
        for j in range(i, L + 1):
            z[i, j] = pattern[(j // (i + 1)) % len(pattern)]
    return z[:, 1:]

In [ ]:
pattern_array([0, 1, 0, -1], 8)

#### function def

In [ ]:
def apply_phase(input_signal, arr):
    # LOGGER.debug(f'input_signal = {input_signal}')
    # LOGGER.debug(f'input_signal.shape = {np.array(input_signal).shape}')
    # LOGGER.debug(f'arr = {arr}')
    # LOGGER.debug(f'arr.shape = {arr.shape}')
    return abs(arr @ input_signal) % 10

In [ ]:
pa = pattern_array([0, 1, 0, -1], 8)
assert (apply_phase([1, 2, 3, 4, 5, 6, 7, 8], pa) == np.array([4, 8, 2, 2, 6, 1, 5, 8])).all()
assert (apply_phase([4, 8, 2, 2, 6, 1, 5, 8], pa) == np.array([3, 4, 0, 4, 0, 4, 3, 8])).all()
assert (apply_phase([3, 4, 0, 4, 0, 4, 3, 8], pa) == np.array([0, 3, 4, 1, 5, 5, 1, 8])).all()
assert (apply_phase([0, 3, 4, 1, 5, 5, 1, 8], pa) == np.array([0, 1, 0, 2, 9, 4, 9, 8])).all()

In [ ]:
def q_1(input_signal, pattern=None):
    input_signal = np.array(input_signal)
    L = len(input_signal)
    if pattern is None:
        pattern = [0, 1, 0, -1]
    LOGGER.debug('making pattern')
    pa = pattern_array(pattern, L)
    LOGGER.debug('done with pattern')
    # LOGGER.debug(f"pattern = {pattern}")
    # LOGGER.debug(f"L = {L}")
    # LOGGER.debug(f"pa = {pa}")
    # LOGGER.debug(f"pa.shape = {pa.shape}")
    
    for phase_num in tqdm.trange(100, leave=False):
        # LOGGER.debug(f"input_signal = {input_signal}")
        new_input = apply_phase(input_signal, pa)
        input_signal = new_input
        
    return int(''.join([str(int(_)) for _ in input_signal[:8]]))


#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_0) == a0
    assert q_1(test_1) == a1
    assert q_1(test_2) == a2
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

note: over `L / 2`, the matrix is defined as `pa(i, j) = int(i < j)`

In [ ]:
LOGGER.setLevel(logging.INFO)
logging.getLogger('matplotlib').setLevel(logging.INFO)

%matplotlib inline

pa = pattern_array([0, 1, 0, -1], 100)

import matplotlib.pyplot as plt

plt.matshow(pa)
plt.colorbar()

this means that for elements L/2 and beyond in the ouptut vector of the apply step are computed as

```python
input_vec[i:].sum().abs() % 10
```

In [ ]:
def q_2(input_signal):
    z_full = np.array(input_signal * 10_000)
    offset = int(''.join(str(_) for _ in z_full[:7]))
    LOGGER.debug(f'offset = {offset}')
    z = z_full[offset:]
    
    for i in range(100):
        z = (z.sum() - z.cumsum() + z) % 10
    
    return int(''.join([str(_) for _ in z[:8]]))

#### tests

In [ ]:
test_0 = [int(_) for _ in '03036732577212944063491565474664']
answer_0 = 84462026

test_1 = [int(_) for _ in '02935109699940807407585447034323']
answer_1 = 78725270

test_2 = [int(_) for _ in '03081770884921959731165446850517']
answer_2 = 53553731

tests = [(test_0, answer_0),
         (test_1, answer_1),
         (test_2, answer_2),]

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    for t, a in tests:
        assert q_2(t) == a
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin